# Scraping

Obtener todos los datos a base de Scrapeo:

### Fases
- Con Selenium, obtener todas las urls de los supermercados
- Con Selenium, utilizar las URLs obtenidas, para sacar las urls de:
    - Aceite girasol
    - Aceite de oliva
    - Leche
- Con selenium, sacar las URLs de las subcategorias de:
    - Aceite de oliva
        - Suave e intenso
        - Virgen
        - Virgen extra.
    - Leche
        - Enriquecida
        - Entera, semi o desnatada
        - Sin Lactosa
    De forma que, por supermercado, nos quedaremos con:
    - URL Aceite Girasol
    - URL Aceite de Oliva Suave e Intenso
    - URL Aceite de Oliva Virgen
    - URL Aceite de Oliva Virgen extra
    - URL leche enriquecida
    - URL leche entera,semi o desnatada
    - URL leche sin lactosa

Con estas URL utilizaremos BEAUTIFUL SOUP para sacar las URL del histórico de cada producto

In [1]:
# Importamos las librerías que necesitamos

# Librerías para automatización de navegadores (Selenium)
# -----------------------------------------------------------------------
from selenium import webdriver                   # Control automático del navegador para realizar scraping web

from selenium.webdriver.chrome.service import Service  # Maneja el servicio de ChromeDriver
from selenium.webdriver.common.by import By      # Para seleccionar elementos en la página web (DOM)
from selenium.webdriver.chrome.options import Options  # Configuración de opciones del navegador (headless mode, etc.)
import time                                      # Para gestionar pausas y temporización

# Librerías para extracción de datos y scraping
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup  # Para analizar y extraer datos de HTML y XML (scraping web)

# Librerías para tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Para manipulación de estructuras de datos como DataFrames
import numpy as np              # Para cálculos numéricos y manejo de arrays
import datetime                 # Para manipulación de fechas y tiempos

# Manejo de tiempo y solicitudes con espera
# -----------------------------------------------------------------------
from time import sleep          # Para pausar la ejecución del código por un tiempo determinado
import random                   # Para generar valores aleatorios, útil para espaciar solicitudes y evitar bloqueos
import requests                 # Repetido (puedes eliminar este segundo import de requests)
# Importación de módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")  # Añadir rutas externas para poder importar módulos desde carpetas superiores
from src.SupportScraping import obtener_url_supermercados,obtener_url_productos_por_supermercado,obtener_urls_por_super_producto_y_categoria,obtener_href_productos# Funciones personalizadas para obtener datos de vuelos

In [2]:
xpaths = ["/html/body/section[1]/div/div[2]/div[1]/div/div[2]",
          "/html/body/section[1]/div/div[2]/div[2]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[3]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[4]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[5]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[6]/div/div[2]/div"]

url_supermercados = [obtener_url_supermercados(xpath) for xpath in xpaths]

In [ ]:
url_supermercados


In [2]:
url_supermercados = ['https://super.facua.org/mercadona/',
                    'https://super.facua.org/carrefour/',
                    'https://super.facua.org/eroski/',
                    'https://super.facua.org/dia/',
                    'https://super.facua.org/hipercor/',
                    'https://super.facua.org/alcampo/']

In [4]:
url_productos = [obtener_url_productos_por_supermercado(url) for url in url_supermercados]

In [ ]:
url_productos

In [2]:
url_productos = [[
'https://super.facua.org/mercadona/aceite-de-girasol/',
  'https://super.facua.org/mercadona/aceite-de-oliva/',
  'https://super.facua.org/mercadona/leche/'],
 ['https://super.facua.org/carrefour/aceite-de-girasol/',
  'https://super.facua.org/carrefour/aceite-de-oliva/',
  'https://super.facua.org/carrefour/leche/'],
 ['https://super.facua.org/eroski/aceite-de-girasol/',
  'https://super.facua.org/eroski/aceite-de-oliva/',
  'https://super.facua.org/eroski/leche/'],
 ['https://super.facua.org/dia/aceite-de-girasol/',
  'https://super.facua.org/dia/aceite-de-oliva/',
  'https://super.facua.org/dia/leche/'],
 ['https://super.facua.org/hipercor/aceite-de-girasol/',
  'https://super.facua.org/hipercor/aceite-de-oliva/',
  'https://super.facua.org/hipercor/leche/'],
 ['https://super.facua.org/alcampo/aceite-de-girasol/',
  'https://super.facua.org/alcampo/aceite-de-oliva/',
  'https://super.facua.org/alcampo/leche/']]

In [3]:
urls_super_category = [obtener_urls_por_super_producto_y_categoria(url_productos[i][0],url_productos[i][1],url_productos[i][2]) for i,p in enumerate(url_productos)]

In [4]:
backup = urls_super_category

In [5]:
supermercados_dfs = [obtener_href_productos(df) for df in urls_super_category]

In [11]:
backup = supermercados_dfs

In [12]:
df = pd.DataFrame()
lista_dfs = []
for i in range(0,len(supermercados_dfs)):
    for e in range(0,len(supermercados_dfs[i])):
        df_temp = supermercados_dfs[i][e]
        df = pd.concat([df,df_temp])
    
    lista_dfs.append(df)

In [23]:
df = lista_dfs[5]

In [27]:
df.sample(6)

,supermercado,categoria,Producto,URL
7,eroski,leche-entera-semi-desnatada,"Leche Desnatada Asturiana, Brik 50 Cl",https://super.facua.org/eroski/leche/leche-des...
179,hipercor,aceite-de-oliva-virgen-extra,Oro Bailen Reserva Familiar Aceite De Oliva Vi...,https://super.facua.org/hipercor/aceite-de-oli...
11,carrefour,aceite-de-oliva-virgen-extra,Aceite De Oliva Virgen Extra Carbonell 5 L.,https://super.facua.org/carrefour/aceite-de-ol...
10,eroski,aceite-de-oliva-virgen-extra,"Aceite De Oliva Virgen Extra Borges, Botella 1...",https://super.facua.org/eroski/aceite-de-oliva...
4,mercadona,leche-entera-semi-desnatada,Leche Desnatada Hacendado 9 L.,https://super.facua.org/mercadona/leche/leche-...
1,alcampo,aceite-de-girasol-,Capicua Aceite De Girasol Alto Oleico Botella ...,https://super.facua.org/alcampo/aceite-de-gira...


In [32]:
df.reset_index(drop=True,inplace=True)

In [33]:
df.to_csv("../datos/backups/URLS_historico_productos.csv")

In [34]:
df = pd.read_csv("../datos/backups/URLS_historico_productos.csv",index_col="Unnamed: 0")

In [35]:
df

,supermercado,categoria,Producto,URL
0,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",https://super.facua.org/mercadona/aceite-de-gi...
1,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",https://super.facua.org/mercadona/aceite-de-gi...
2,mercadona,aceite-de-oliva-suave-e-intenso,Aceite De Oliva Intenso Hacendado 3 L.,https://super.facua.org/mercadona/aceite-de-ol...
3,mercadona,aceite-de-oliva-suave-e-intenso,Aceite De Oliva Suave Hacendado 3 L.,https://super.facua.org/mercadona/aceite-de-ol...
4,mercadona,aceite-de-oliva-virgen,Aceite De Oliva Virgen Hacendado 1 L.,https://super.facua.org/mercadona/aceite-de-ol...
...,...,...,...,...
1526,alcampo,leche-sin-lactosa,"Puleva Leche Semidesnatada De Vaca, Sin Lactos...",https://super.facua.org/alcampo/leche/puleva-l...
1527,alcampo,leche-sin-lactosa,"Puleva Omega 3 Preparado Lacteo Desnatado, Si...",https://super.facua.org/alcampo/leche/puleva-o...
1528,alcampo,leche-sin-lactosa,"Puleva Omega 3 Preparado Lacteo Desnatado, Sin...",https://super.facua.org/alcampo/leche/puleva-o...
1529,alcampo,leche-sin-lactosa,Puleva Proteína Extra Pro Leche De Vaca Desnat...,https://super.facua.org/alcampo/leche/puleva-p...
